# Web Scraping & Pagination

## EIA Forms 923/906

### Setup

In [2]:
import bs4
import requests

In [3]:
eia_923_url = "https://www.eia.gov/electricity/data/eia923/"

### Example: getting links from a webpage

We'll be looking at the [EIA 923 webpage](https://www.eia.gov/electricity/data/eia923/) in examples and the [EIA-906 historical archive](https://www.eia.gov/electricity/data/eia923/eia906u.php) in exercises.

In [ ]:
eia_923_response = requests.get(eia_923_url)
eia_923_response.text

In [ ]:
eia_923_soup = bs4.BeautifulSoup(eia_923_response.text)
eia_923_soup

In [ ]:
eia_923_soup.find_all("title")

In [ ]:
eia_923_all_a_tags = eia_923_soup.find_all("a")
eia_923_all_a_tags

In [ ]:
eia_923_a_hrefs = eia_923_soup.find_all("a", href=True)
eia_923_a_hrefs

In [ ]:
eia_923_zip_tags = []
for a in eia_923_a_hrefs:
    if a["href"].lower().endswith(".zip") and "f923" in a["href"]:
        eia_923_zip_tags.append(a)
eia_923_zip_tags

### Challenge: get all the relevant `a` tags from EIA 906

Lots of the data that is collected in EIA 923 was collected in EIA 906 in the past.

We'll have you work through the scraping steps on the 906 data to get a sense of how this all works.


Let's get the relevant `a` tags from the [EIA 906 page](https://www.eia.gov/electricity/data/eia923/eia906u.php):

Start with the skeleton code outlined below - we expect a variable called `eia_906_xls_tags` at the end, which holds all the tags that refer to the actual 1970-2000 data files.

### Solution

In [ ]:
eia_906_url = "https://www.eia.gov/electricity/data/eia923/eia906u.php"
# get the page contents
# turn it into a collection of tags
# filter them down to the tags that contain the links to XLS data - for all years 1970-2000

eia_906_xls_tags = []

In [6]:
eia_906_response = requests.get(eia_906_url)
eia_906_soup = bs4.BeautifulSoup(eia_906_response.text)

eia_906_a_hrefs = eia_906_soup.find_all("a", href=True)
eia_906_xls_tags = []
for a in eia_906_a_hrefs:
    if ".xls" in a["href"].lower():
        eia_906_xls_tags.append(a)

### Example: downloading data

In [7]:
eia_906_one_link = eia_906_xls_tags[0]
eia_906_one_link

<a href="/electricity/data/eia923/archive/xls/utility/f7592000mu.xls" title="2000M  ">
XLS</a>

In [8]:
eia_906_one_df = requests.get(eia_906_one_link["href"])

MissingSchema: Invalid URL '/electricity/data/eia923/archive/xls/utility/f7592000mu.xls': No scheme supplied. Perhaps you meant https:///electricity/data/eia923/archive/xls/utility/f7592000mu.xls?

In [ ]:
from urllib.parse import urljoin

In [ ]:
eia_906_one_full_url = urljoin(eia_906_url, eia_906_one_link["href"])

In [ ]:
eia_906_one_response = requests.get(eia_906_one_full_url)

In [ ]:
import pandas as pd

eia_906_one_df = pd.read_excel(eia_906_one_response.content)

In [ ]:
eia_906_one_df = pd.read_excel(eia_906_one_full_url)

### Challenge: get the Form 906 file contents

OK, so now we know how to scrape a bunch of URLs from a webpage. Let's read the Form 906 files into our program! Since they're XLS files, we can read them directly from a URL using `pandas.read_excel`.

Try making a list, `eia_906_dataframes`, that includes all of the data files from the [EIA 906 page](https://www.eia.gov/electricity/data/eia923/eia906u.php) - start with the (minimal) scaffold below!

### Solution

In [ ]:
import pandas as pd

eia_906_dataframes = []

# loop through the eia_906_xls_tags and make a pd.DataFrame for each one

### Extension: combining the data

### Discussion

Why might you choose to do all this instead of just manually collecting links?

## Pagination

### Setup

In [ ]:
eia_api_base_url = "https://api.eia.gov/v2/electricity"
api_key = "3zjKYxV86AqtJWSRoAECir1wQFscVu6lxXnRVKG8"

### Example: getting the first few pages

In [ ]:
first_page = requests.get(
    f"{eia_api_base_url}/facility-fuel/data",
    params={
        "data[]": "generation",
        "facets[state][]": "PR",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "sort[1][column]": "plantCode",
        "sort[1][direction]": "desc",
        "api_key": api_key
    }
).json()["response"]

In [ ]:
first_page.keys()

In [ ]:
pd.DataFrame(first_page["data"])

In [ ]:
first_page["warnings"]

> Offset stipulates the row number the API should begin its return with, out of all the eligible rows our query would otherwise provide.
>
> [...]
>
> `https://api.eia.gov/v2/electricity/retail_sales/data?api_key=xxxxxx&data[]=price&facets[sectorid][]=RES&facets[stateid][]=CO&frequency=monthly&sort[0][column]=period&sort[0][direction]=desc&offset=24`
>
> In the above example, the API will skip over the first 24 eligible rows (offset=24), which translates into 24 months (frequency=monthly).

source: https://www.eia.gov/opendata/documentation.php

In [ ]:
common_params = {
    "data[]": "generation",
    "facets[state][]": "CO",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "sort[1][column]": "plantCode",
    "sort[1][direction]": "desc",
    "api_key": api_key
}

In [ ]:
next_page = requests.get(
    f"{eia_api_base_url}/facility-fuel/data", params=common_params | {"offset": 5000}
).json()["response"]

In [ ]:
pd.DataFrame(next_page["data"])

In [ ]:
for i in range(5):
    print(i)

In [ ]:
for i in range(10, 15):
    print(i)

In [ ]:
for i in range(0, 15, 5):
    print(i)

### Challenge: `range`

If you wanted to set a fresh `offset` for every page in a number of rows, how would you do that? Imagine there are 23,456 rows and each page must be 5000 rows long.

### Solution

In [ ]:
total_rows = 23_456
page_size = 5000

# print out the offset for each page - should be something like 0, 100, ... 1100, 1200

### Challenge: pagination

OK, now let's put it all together! 

Let's try to get the net generation data in Puerto Rico that is in the EIA API.

Instead of getting all of it, which will take a lot of waiting around, let's just grab the first 12,345 rows.

Start with the following code and modify it to work:

### Solution

In [ ]:
all_records = []
# loop through the necessary pages to get 12,345 rows
    print(f"Getting page starting at {offset}...")
    page = requests.get(
      f"{eia_api_base_url}/facility-fuel/data",
      params={
          # what goes here?
      }
    ).json()["response"]
    all_records.append(pd.DataFrame(page["data"]))

df = pd.concat(all_records)

### Extension: getting *all* the rows

## Key points

- beautiful soup lets you grab links out of a webpage so that you can then download them
- if you need to get more than one request worth of results from an API, they usually provide some "pagination" capabilities so you can make all the requests programmatically.
- web scraping is a wide world - if you get stuck, try searching for some of the keywords above.